In [31]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader,Dataset
import alive_progress
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.globals import CurrentConfig

In [32]:
path=r"E:\科研\ESV计算\Sri Lanka"
yield_c=pd.read_csv(path+r"\yield_c.csv")
pp_c=pd.read_csv(path+r"\producerPrices_c.csv")
x=pp_c[pp_c['Item']=="Maize (corn)"]['Value']
x=torch.FloatTensor(x).view(-1)

In [33]:
# 设置迭代器
class MyDataSet(Dataset):
    def __init__(self,seq,ws=6):
        self.ori=[i for i in seq[:ws]]
        self.label=[i for i in seq[ws:]]
        self.reset()
        self.ws=ws

    def set(self,dpi):
        self.x.append(dpi)
    def reset(self):
        self.x=self.ori[:]
    def get(self,idx):
        return self.x[idx:idx+self.ws],self.label[idx]
    def __len__(self):
        return len(self.x)

In [34]:
   
class Net3(nn.Module):
    def __init__(self,in_features=54,n_hidden1=128,n_hidden2=256,n_hidden3=512,out_features=7):
        super(Net3, self).__init__()
        self.flatten=nn.Flatten()
        self.hidden1=nn.Sequential(
            nn.Linear(in_features,n_hidden1,False),
           
            nn.ReLU()
        )
        self.hidden2=nn.Sequential(
            nn.Linear(n_hidden1,n_hidden2),

            nn.ReLU()
        )
        self.hidden3=nn.Sequential(
            nn.Linear(n_hidden2,n_hidden3),

            nn.ReLU()
        )
        self.out=nn.Sequential(nn.Linear(n_hidden3,out_features))

    def forward(self,x):
        x=self.flatten(x)
        x=self.hidden2(self.hidden1(x))
        x=self.hidden3(x)
        return self.out(x)



class CNN(nn.Module):
    def __init__(self, output_dim=1,ws=6):
        super(CNN, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv1d(ws, 64, 1)
        self.lr = nn.LeakyReLU(inplace=True)
        self.conv2 = nn.Conv1d(64, 128, 1)

        self.bn1, self.bn2 = nn.BatchNorm1d(64), nn.BatchNorm1d(128)
        self.bn3, self.bn4 = nn.BatchNorm1d(1024), nn.BatchNorm1d(128)
        self.flatten = nn.Flatten()
        self.lstm1 = nn.LSTM(128, 1024)
        self.lstm2 = nn.LSTM(1024, 256)
        self.lstm3=nn.LSTM(256,512)
        self.fc = nn.Linear(512, 512)
        self.fc4=nn.Linear(512,256)
        self.fc1 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, output_dim)

    @staticmethod
    def reS(x):
        return x.reshape(-1, x.shape[-1], x.shape[-2])

    def forward(self, x):
        x = self.reS(x)
        x = self.conv1(x)  # torch.Size([1, 64, 1])
        x = self.lr(x)

        x = self.conv2(x)  # torch.Size([1, 128, 32])
        x = self.lr(x)

        # x = self.conv3(x)  # torch.Size([32, 300, 298])
        # x = self.maxpool3(x)  # torch.Size([32, 300, 100])
        x = self.flatten(x)
        # 注意Flatten层后输出为(N×T,C_new)，需要转换成(N,T,C_new)

        # LSTM部分
        x, h = self.lstm1(x)
        x, h = self.lstm2(x)
        x,h=self.lstm3(x)
        # 注意这里只使用隐层的输出
        x, _ = h

        x = self.fc(x.reshape(-1, ))
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x


In [35]:
def Train(model,train_data,seed=1):
    device="cuda" if torch.cuda.is_available() else "cpu"
    model=model.to(device)
    Mloss=100000
    path=r"E:\科研\ESV计算\Sri Lanka\Train\MODEL_\%s.pth"%seed
    # 设置损失函数,这里使用的是均方误差损失
    criterion = nn.MSELoss()
    # 设置优化函数和学习率lr
    optimizer=torch.optim.Adam(model.parameters(),lr=1e-5,betas=(0.9,0.99),
                               eps=1e-07,weight_decay=0)
    # 设置训练周期
    epochs =3000
    criterion=criterion.to(device)
    model.train()
    for epoch in range(epochs):
        total_loss=0

        for i in range(len(x)-6):
            # 每次更新参数前都梯度归零和初始化
            seq,y_train=train_data.get(i)
            seq,y_train=torch.FloatTensor(seq),torch.FloatTensor([y_train])
            seq=seq.unsqueeze(dim=0)
            seq,y_train=seq.to(device),y_train.to(device)

            optimizer.zero_grad()
            # 注意这里要对样本进行reshape，
            # 转换成conv1d的input size（batch size, channel, series length）
            y_pred = model(seq)
            loss = criterion(y_pred, y_train)
            loss.backward()
            train_data.set(y_pred.to("cpu").item())
            optimizer.step()
            total_loss+=loss

        train_data.reset()
        if total_loss.tolist()<Mloss:
            Mloss=total_loss.tolist()
            torch.save(model.state_dict(),path)
            print("Saving")
        print(f'Epoch: {epoch+1:2} Mean Loss: {total_loss.tolist()/len(train_data):10.8f}')
    return model

In [36]:
ws=6

In [37]:
train_data=MyDataSet(x,ws)
d=CNN(ws=ws)
# Train(d,train_data,4)

In [38]:
checkpoint=torch.load(r"E:\科研\ESV计算\Sri Lanka\Train\MODEL_\3.pth")
d.load_state_dict(checkpoint)
d.to("cpu")

CNN(
  (relu): ReLU(inplace=True)
  (conv1): Conv1d(6, 64, kernel_size=(1,), stride=(1,))
  (lr): LeakyReLU(negative_slope=0.01, inplace=True)
  (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lstm1): LSTM(128, 1024)
  (lstm2): LSTM(1024, 256)
  (lstm3): LSTM(256, 512)
  (fc): Linear(in_features=512, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [41]:
pp_c

,Unnamed: 0,Year,Value,Item,Unit
0,10,2001,14.6500,Maize (corn),卢比/kg
1,11,2002,18.1100,Maize (corn),卢比/kg
2,12,2003,19.5400,Maize (corn),卢比/kg
3,13,2004,21.2700,Maize (corn),卢比/kg
4,14,2005,20.5800,Maize (corn),卢比/kg
5,15,2006,20.1500,Maize (corn),卢比/kg
6,16,2007,24.7100,Maize (corn),卢比/kg
7,17,2008,36.7300,Maize (corn),卢比/kg
8,18,2009,35.4600,Maize (corn),卢比/kg
9,19,2010,34.6400,Maize (corn),卢比/kg


In [48]:
pre_data=pp_c[pp_c['Item']=='Sorghum']['Value'].tolist()

In [51]:
l=pre_data[:]
for i in range(len(x)-ws+1):
    l.append(d(torch.FloatTensor(l[-ws:]).unsqueeze(dim=0)).item())

In [52]:
L=Line()
L.add_xaxis([i for i in range(len(x))])
L.add_yaxis("Pred",l)
L.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
L.set_global_opts(title_opts=opts.TitleOpts(title='sorghum production forecasts')
                            
                             )

L.render_notebook()

In [13]:
t=pp_c[pp_c['Item']=="Millet"]['Value'].tolist()

In [17]:
s=t[:ws]
for i in range(len(x)-ws+1):
    s.append(d(torch.FloatTensor(s[-ws:]).unsqueeze(dim=0)).item())

In [18]:
l=Line()
l.add_xaxis([i for i in range(len(x))])
l.add_yaxis("Pred Data",s)
l.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
l.set_global_opts(title_opts=opts.TitleOpts(title='LSTM CNN'))

l.render_notebook()

In [23]:
sum(s)/len(s)

55.14148219153995

In [24]:
d=pd.read_csv(path+r"\lstmPre_producerPrices.csv")

In [26]:
d.groupby("Item").mean()['Value']

C:\Users\lenovo\AppData\Local\Temp\ipykernel_19776\1082947277.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  d.groupby("Item").mean()['Value']


Item
Maize (corn)     34.713610
Millet          117.373157
Rice             31.194733
Sorghum          67.324718
Name: Value, dtype: float64

In [27]:
s

[40.22,
 46.63,
 37.75,
 39.76,
 43.33,
 40.00455093383789,
 55.848575592041016,
 44.210201263427734,
 50.83948516845703,
 65.7563247680664,
 47.99384307861328,
 87.69588470458984,
 34.56197738647461,
 65.54164123535156,
 77.4919662475586,
 52.897911071777344,
 103.76702880859375,
 18.202669143676758,
 74.86048889160156,
 71.91515350341797,
 58.693424224853516]